In [3]:
from scipy.spatial.transform import Rotation
import numpy as np

# Sensor Models
Rigorous sensor models (think ISIS and USGSCSM) are fundamentally a bunch of transformations between reference frames and coordinate systems. When you project a pixel to a ground point in ISIS, the sensor model is doing the following transformations:

1. Transform the image coordinate into a detector coordinate
1. Transform the detector coordinate into a distorted focal plane coordinate
1. Transform the distorted focal plane coordinate into an undistorted focal plane coordinate
1. Transform the undistorted focal plane coordinate into a look vector in the sensor reference frame
1. Transform the look vector in the sensor reference frame into a look vector in the target reference frame

Rotations largely come into during the final transformation, in particular the rotation from the sensor reference frame to the target reference frame.

## Getting From the Sensor Reference Frame To the Target Reference Frame

```
OSIRIS Frame Tree
--------------------------------------

   The diagram below shows the OSIRIS frame hierarchy.


                               "J2000" INERTIAL
           +-----------------------------------------------------+
           |                 |        |        |                 |
           |<-pck        ck->|        |        |<-pck       pck->|
           |                 |        |        |                 |
           V                 V        |        V                 V
       "67P/C-G_FIXED"  "67P/C-G_CK"  | "LUTETIA_FIXED"   "STEINS_FIXED"
         COMET BFXD      COMET BFXD   |  ASTEROID BFXD    ASTEROID BFXD
       ---------------  ------------  | --------------    --------------
                                      |
                                      |<-ck
                                      |
                                      V
                               "ROS_SPACECRAFT"
           +-----------------------------------------------------+
           |                 |                 |                 |
           |                 |<-fixed          |<-fixed          |
           |                 |                 |                 |
           |                 V                 V                 |
           |   "ROS_OSIRIS_NAC_URF"      "ROS_OSIRIS_WAC_URF"    |
           |   --------------------      --------------------    |
           |                                                     |
           |<-fixed                                              |<-fixed
           |                                                     |
           V                                                     V
     "ROS_OSIRIS_NAC"                                     "ROS_OSIRIS_WAC"
     ----------------                                      ----------------
```

Taken from the [Rosetta Frame Kernel](https://naif.jpl.nasa.gov/pub/naif/ROSETTA/kernels/fk/ROS_V33.TF)

For most sensor models, getting from the sensor reference frame to the target reference frame is a 3-step process:

1. Rotate to the spacecraft reference frame
1. Rotate to the J2000 reference frame
1. Rotate to the target reference frame

<span style="color:blue">Rotate the look vector (1.2, 1.6, 50) from the sensor reference frame to the target reference frame based on the following information:</span>

* The rotation from the sensor reference frame to the spacecraft reference frame is a 90 degree rotation around the X-axis followed by a 90 degree rotation around the Y-axis
* The rotation from the spacecraft reference frame to the J2000 reference frame is defined by the quaternion $\frac{1}{\sqrt{2}}+\frac{1}{\sqrt{2}}i+0j+0k$
* The rotation from the J2000 reference frame to the target reference frame is defined by the quaternion $\frac{1}{\sqrt{2}}+0i+0j+\frac{1}{\sqrt{2}}k$

In [9]:
sensor_look = np.array([1.2, 1.6, 50.0])

sensor_to_spacecraft = Rotation.from_euler('xyz', [90, 90, 0], degrees=True)
spacecraft_to_j2000 = Rotation.from_quat([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])
j2000_to_target = Rotation.from_quat([0, 0, 1/np.sqrt(2), 1/np.sqrt(2)])

# Hint, you can chain scipy rotations together by left multiplication
sensor_to_target = j2000_to_target * spacecraft_to_j2000 * sensor_to_spacecraft

target_look = sensor_to_target.apply(sensor_look)

print('The rotated look vector is', target_look)

The rotated look vector is [ -1.2   1.6 -50. ]


<span style="color:blue">Using the same rotations as before, rotate the look vector (1, 0, 0) from the target reference frame to the sensor reference frame</span>

In [12]:
new_target_look = np.array([1, 0, 0])
target_to_sensor = sensor_to_target.inv()
new_sensor_look = target_to_sensor.apply(new_target_look)

print('The rotated look vector is', new_sensor_look)

The rotated look vector is [-1.00000000e+00  4.44089210e-16  4.93038066e-32]
